In [50]:
import pandas as pd
import plotly.graph_objects as go
import networkx as nx

def create_gene_network(df,color):

    # 计算相关性
    correlation_matrix = df.transpose().corr()

    # 使用networkx创建一个网络图
    G = nx.Graph()
    for gene1 in df.index:
        for gene2 in df.index:
            if gene1 != gene2:
                G.add_edge(gene1, gene2, weight=correlation_matrix.loc[gene1, gene2])

    # 为了可视化，我们可能只想显示高相关性的基因对
    threshold = 0.6
    edges = [(u, v) for (u, v, d) in G.edges(data=True) if abs(d['weight']) > threshold]
    G = G.edge_subgraph(edges).copy()  # 使用edges创建一个新的图，并使用copy()避免状态问题

    # 生成节点的位置
    pos = nx.spring_layout(G)  # 所有节点的位置

    # 将位置作为节点属性添加到G中
    for node in G.nodes():
        G.nodes[node]['pos'] = pos[node]

    # 使用plotly创建网络图
    edge_x = []
    edge_y = []
    for edge in G.edges():
        x0, y0 = G.nodes[edge[0]]['pos']
        x1, y1 = G.nodes[edge[1]]['pos']
        edge_x.append(x0)
        edge_x.append(x1)
        edge_x.append(None)
        edge_y.append(y0)
        edge_y.append(y1)
        edge_y.append(None)

    edge_trace = go.Scatter(
        x=edge_x, y=edge_y,
        line=dict(width=0.5, color='#888'),
        hoverinfo='none',
        mode='lines')

    node_x = []
    node_y = []
    for node in G.nodes():
        x, y = G.nodes[node]['pos']
        node_x.append(x)
        node_y.append(y)

    node_trace = go.Scatter(
        x=node_x, y=node_y,
        mode='markers',
        hoverinfo='text',
        marker=dict(
            showscale=True,
            colorscale=color,
            size=10,
            colorbar=dict(
                thickness=15,
                title='Node Connections',
                xanchor='left',
                titleside='right'
            ),
            line_width=2))

    node_adjacencies = []
    node_text = []
    for node, adjacencies in enumerate(G.adjacency()):
        node_adjacencies.append(len(adjacencies[1]))
        node_text.append(adjacencies[0])

    node_trace.marker.color = node_adjacencies
    node_trace.text = node_text

    fig = go.Figure(data=[edge_trace, node_trace],
                    layout=go.Layout(
                        title='Gene Co-expression Network',
                        showlegend=False,
                        hovermode='closest',
                        margin=dict(b=0, l=0, r=0, t=40),
                        annotations=[
                            dict(
                                text="",
                                showarrow=False,
                                xref="paper", yref="paper",
                                x=0.005, y=-0.002)
                        ],
                        xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
                        yaxis=dict(showgrid=False, zeroline=False, showticklabels=False))
                    )

    # 定义fig的布局，设置宽度和高度
    fig.update_layout(
        autosize=False,
        width=800, 
        height=600,
        template="plotly_white"
    )

    fig.show()

# 调用函数
df = pd.read_csv('./demo-fpkm.csv', index_col=0)
create_gene_network(df,'Portland')

In [ ]:
import pandas as pd
import plotly.graph_objects as go
import networkx as nx

def create_gene_network(df,color,bubble_size_ratio):

    # 计算相关性
    correlation_matrix = df.transpose().corr()

    # 使用networkx创建一个网络图
    G = nx.Graph()
    for gene1 in df.index:
        for gene2 in df.index:
            if gene1 != gene2:
                G.add_edge(gene1, gene2, weight=correlation_matrix.loc[gene1, gene2])

    # 为了可视化，我们可能只想显示高相关性的基因对
    threshold = 0.5 # 相关性系数
    edges = [(u, v) for (u, v, d) in G.edges(data=True) if abs(d['weight']) > threshold]
    G = G.edge_subgraph(edges).copy()  # 使用edges创建一个新的图，并使用copy()避免状态问题

    # 生成节点的位置
    pos = nx.spring_layout(G)  # 所有节点的位置

    # 将位置作为节点属性添加到G中
    for node in G.nodes():
        G.nodes[node]['pos'] = pos[node]

    # 使用plotly创建网络图
    edge_x = []
    edge_y = []
    for edge in G.edges():
        x0, y0 = G.nodes[edge[0]]['pos']
        x1, y1 = G.nodes[edge[1]]['pos']
        edge_x.append(x0)
        edge_x.append(x1)
        edge_x.append(None)
        edge_y.append(y0)
        edge_y.append(y1)
        edge_y.append(None)

    edge_trace = go.Scatter(
        x=edge_x, y=edge_y,
        line=dict(width=0.5, color='#888'),
        hoverinfo='none',
        mode='lines')

    node_x = []
    node_y = []
    for node in G.nodes():
        x, y = G.nodes[node]['pos']
        node_x.append(x)
        node_y.append(y)

    node_trace = go.Scatter(
        x=node_x, y=node_y,
        mode='markers',
        hoverinfo='text',
        marker=dict(
            showscale=True,
            colorscale=color,
            # 移除固定大小
            # size=10,  # 这行不再需要
            colorbar=dict(
                thickness=15,
                title='Node Connections',
                xanchor='left',
                titleside='right'
            ),
            line_width=2))

    node_adjacencies = []
    node_text = []
    node_sizes = []  # 添加一个列表来存储基于节点连接数的大小
    for node, adjacencies in enumerate(G.adjacency()):
        node_adjacencies.append(len(adjacencies[1]))
        node_text.append(adjacencies[0])
        node_degree = len(adjacencies[1])
        scaled_size = 10 + (node_degree * bubble_size_ratio)  # 基础大小为5，每个连接增加5个单位大小
        node_sizes.append(scaled_size)

    node_trace.marker.color = node_adjacencies
    node_trace.text = node_text
    node_trace.marker.size = node_sizes  # 更新marker的大小

    
    fig = go.Figure(data=[edge_trace, node_trace],
                    layout=go.Layout(
                        title='Gene Co-expression Network',
                        showlegend=False,
                        hovermode='closest',
                        margin=dict(b=0, l=0, r=0, t=40),
                        annotations=[
                            dict(
                                text="",
                                showarrow=False,
                                xref="paper", yref="paper",
                                x=0.005, y=-0.002)
                        ],
                        xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
                        yaxis=dict(showgrid=False, zeroline=False, showticklabels=False))
                    )

    # 定义fig的布局，设置宽度和高度
    fig.update_layout(
        autosize=False,
        width=800, 
        height=600,
        template="plotly_white"
    )

    fig.show()


# 调用函数
df = pd.read_csv('./demo-fpkm.csv', index_col=0)
create_gene_network(df,'Portland',5)

这张图是一个基因共表达网络，表示了基因之间的相关性。这种网络用于展示基因的活性是如何相互关联的，这有助于识别可能共同参与某一生物过程的基因集合。这里是这张图的一些关键特点：

节点（Nodes）：每一个圆圈代表一个基因。例如，图中标记为 "Mt15" 的节点代表名为 "Mt15" 的基因。

边（Edges）：连线表示两个基因之间的相关性。在此网络图中，一个边的存在意味着两个基因之间有高度的共表达关系。

节点颜色：颜色代表节点的连接数，也叫“度”。深色的节点有更多的连接，而浅色的节点则有较少的连接。这意味着深色的节点在该共表达网络中与更多的基因高度相关。

节点大小：所有节点似乎都有相同的大小。但在某些网络图中，节点的大小可能会变化以表示某种度量，例如基因的表达量。

颜色条（Colorbar）：图的右边有一个颜色条，显示了节点颜色与其连接数之间的关系。例如，深蓝色代表有14个连接的节点。

从这张图可以看出，有些基因（节点）与许多其他基因高度相关，而有些基因只与少数几个基因相关。具有很多连接的基因可能在某些生物过程中起到核心作用，因为它们与许多其他基因都有关联。

总之，这种图为研究者提供了一个直观的方法，可以看到大量基因数据中的模式和关系，并可能帮助研究者识别重要的基因和基因之间的相互作用。

In [3]:
import pandas as pd
import plotly.graph_objects as go
import networkx as nx

def create_gene_network(df,color,bubble_size_ratio,k_value,iterations_value):

    # 计算相关性
    correlation_matrix = df.transpose().corr()

    # 使用networkx创建一个网络图
    G = nx.Graph()
    for gene1 in df.index:
        for gene2 in df.index:
            if gene1 != gene2:
                G.add_edge(gene1, gene2, weight=correlation_matrix.loc[gene1, gene2])

    threshold = 0.5 # 相关性系数
    edges = [(u, v) for (u, v, d) in G.edges(data=True) if abs(d['weight']) > threshold]
    G = G.edge_subgraph(edges).copy()  # 使用edges创建一个新的图，并使用copy()避免状态问题

    # pos设置,k越小则点越紧
    pos = nx.spring_layout(G, k=k_value, iterations=iterations_value)

    # 将位置作为节点属性添加到G中
    for node in G.nodes():
        G.nodes[node]['pos'] = pos[node]

    # 使用plotly创建网络图
    edge_x = []
    edge_y = []
    for edge in G.edges():
        x0, y0 = G.nodes[edge[0]]['pos']
        x1, y1 = G.nodes[edge[1]]['pos']
        edge_x.append(x0)
        edge_x.append(x1)
        edge_x.append(None)
        edge_y.append(y0)
        edge_y.append(y1)
        edge_y.append(None)
    
    # 分别创建正相关和负相关的边
    edge_x_pos, edge_y_pos = [], []
    edge_x_neg, edge_y_neg = [], []
    
    # 根据权重将边分为正负两组
    for edge in G.edges(data=True):
        x0, y0 = pos[edge[0]]
        x1, y1 = pos[edge[1]]
        if edge[2]['weight'] > 0:
            edge_x_pos.extend([x0, x1, None])
            edge_y_pos.extend([y0, y1, None])
        else:
            edge_x_neg.extend([x0, x1, None])
            edge_y_neg.extend([y0, y1, None])


    edge_trace_pos = go.Scatter(
        x=edge_x_pos, y=edge_y_pos,
        line=dict(width=0.3, color='red'),
        hoverinfo='none',
        mode='lines'
    )
    
    edge_trace_neg = go.Scatter(
        x=edge_x_neg, y=edge_y_neg,
        line=dict(width=0.2, color='blue'),
        hoverinfo='none',
        mode='lines'
    )

    node_x = []
    node_y = []
    for node in G.nodes():
        x, y = G.nodes[node]['pos']
        node_x.append(x)
        node_y.append(y)

    node_trace = go.Scatter(
        x=node_x, y=node_y,
        mode='markers',
        hoverinfo='text',
        marker=dict(
            showscale=True,
            colorscale=color,
            colorbar=dict(
                thickness=15,
                title='Node Connections',
                xanchor='left',
                titleside='right'
            ),
            line_width=2))

    node_adjacencies = []
    node_text = []
    node_sizes = []  # 添加一个列表来存储基于节点连接数的大小
    for node, adjacencies in enumerate(G.adjacency()):
        node_adjacencies.append(len(adjacencies[1]))
        node_text.append(adjacencies[0])
        node_degree = len(adjacencies[1])
        # 设置节点大小
        scaled_size = 15 + (node_degree * bubble_size_ratio)  # 基础大小为5，每个连接增加5个单位大小
        node_sizes.append(scaled_size)

    node_trace.marker.color = node_adjacencies
    node_trace.text = node_text
    node_trace.marker.size = node_sizes  # 更新marker的大小

    fig = go.Figure(data = [edge_trace_pos, edge_trace_neg, node_trace],
                    layout=go.Layout(
                        title='Gene Co-expression Network',
                        showlegend=False,
                        hovermode='closest',
                        margin=dict(b=0, l=0, r=0, t=40),
                        annotations=[
                            dict(
                                text="",
                                showarrow=False,
                                xref="paper", yref="paper",
                                x=0.005, y=-0.002)
                        ],
                        xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
                        yaxis=dict(showgrid=False, zeroline=False, showticklabels=False))
                    )

    # 定义fig的布局，设置宽度和高度
    fig.update_layout(
        autosize=False,
        width=800, 
        height=600,
        template="plotly_white"
    )
    # fig.update_traces(marker_symbol='diamond') # 更改marker的形状

    fig.show()


# 调用函数
df = pd.read_csv('./demo_RPKM.txt', sep='\t', index_col=0)
create_gene_network(df,'Portland',1,0.5,10)